In [3]:
import pandas as pd
from itertools import product
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pickle
pd.set_option('display.max_columns', None)

In [4]:
original = pd.read_csv('../html2023-spring-final-project/train.csv')

In [5]:
trainingLabels = original['Danceability']
original.drop(['Danceability'], axis=1, inplace=True) 

In [6]:
def fillOptions(data, option = 'max'):
    if data.isna().sum() != len(data):
        if option == 'max':
            return data.value_counts().idxmax()
        elif option == 'mean':
            return data.mean()
        elif option == 'median':
            return data.median()

In [7]:
def filterArtistComposerDance(data, nameColumnFill):

    listArtist = data['Artist'].unique()
    listComposer = data['Composer'].unique()
    
    filter = list(product(listArtist, listComposer))

    newData = pd.DataFrame(columns=data.columns)

    for i in filter:
        artist, composer = i[0], i[1]
        filterData = data[(data['Artist'] == artist) & (data['Composer'] == composer) ].copy()
        if len(filterData) != 0:
            # Fill column Name
            for nameColumn in nameColumnFill:
                fillInfo = fillOptions(filterData[nameColumn], option = 'max')
                if fillInfo != None:
                    filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

            newData = pd.concat([newData, filterData], ignore_index=True)
    return newData

In [8]:
def filterArtist(data, nameColumnFill):
    listDance = data['Artist'].unique()
    filter = listDance


    newData = pd.DataFrame(columns=data.columns)

    for i in filter:
        dance = i
        filterData = data[ (data['Artist'] == dance)].copy()
    
    
        if len(filterData) != 0:
            # Fill column Name
            for nameColumn in nameColumnFill:
                fillInfo = fillOptions(filterData[nameColumn], option = 'max')
                if fillInfo != None:
                    filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

            newData = pd.concat([newData, filterData], ignore_index=True)

    return newData

In [9]:
def filterFillData(data, nameColumnFill):
    if not data.isnull().any().any():
        return data
    else:
        for nameColumn in nameColumnFill:
            fillInfo = fillOptions(data[nameColumn], option = 'max')
            if fillInfo != None:
                data.loc[:,nameColumn].fillna(fillInfo, inplace=True)
    return data



In [11]:
def dataPreprocessing(original):
    
    # pd.options.mode.chained_assignment = None

    data = original.copy()

    nameColumnFill = ['Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Duration_ms', 'Views', 'Likes', "Stream" , "Comments"]

    # License and official_video
    data['Licensed'].fillna(data['official_video'], inplace=True)
    data['Licensed'].fillna(False, inplace=True)

    data['official_video'].fillna(data['Licensed'], inplace=True)
    data['official_video'].fillna(False, inplace=True)

    data['official_video'].fillna(False, inplace=True)
    data['Licensed'].fillna(False, inplace=True)

    data['Licensed'] =  data['Licensed'].map({True: 1, False: 0})
    data['official_video'] = data['official_video'].map({True: 1, False: 0})
    
    # Create new class = 'Unknown'
    data['Composer'].fillna("Unknown", inplace=True)
    data['Artist'].fillna("Unknown", inplace=True)
    data['Album_type'].fillna("Unknown", inplace=True)

    newData = filterArtistComposerDance(data, nameColumnFill)
    data = newData.copy()

    newData = filterArtist(data, nameColumnFill)
    data = newData.copy()

    newData = filterFillData(data, nameColumnFill)
    data = newData.copy()

    #Transform type key to use as class
    data['Key'] = data['Key'].astype(int)
    data['Key'] = data['Key'].astype(str)

    data = data.sort_values('id')

    # DELETE Track, Album, Uri, Url_spotify, Url_youtube, Description, Title, Channel, id, Comments
    data.drop(['Track', 'Album', 'Uri', 'Url_spotify', 'Url_youtube', 'Description', 'Title', 'Channel', 'id'], axis=1, inplace=True)

    # pd.options.mode.chained_assignment = 'warn'

    return data
    


In [12]:
def convertEncoderPD(data, prefix = 'key'):
    titleKeys = []
    for i in range(data.shape[1]):
        titleKeys.append(f'{prefix}_{i}')
    
    return pd.DataFrame(data=data, columns= titleKeys)


In [13]:
# minX -60  maxX = 0  ~ 0 - 1
# y = (-1/60) x
def scaleMinMaxLoudness(data):
    return -data/60

In [14]:
def createEncodeDataTraining(data):

    encoderKey = OneHotEncoder()
    encodedKey = encoderKey.fit_transform(data[['Key']])
    Key = encodedKey.toarray()
    key_pd = convertEncoderPD(Key, prefix = 'key')

    encoderAlbumType = OneHotEncoder()
    encodedKeyAlbumType = encoderAlbumType.fit_transform(data[['Album_type']])
    AlbumType = encodedKeyAlbumType.toarray()
    AlbumType_pd = convertEncoderPD(AlbumType, prefix = 'AlbumType')

    encoderComposer = OneHotEncoder()
    encodedKeyComposer = encoderComposer.fit_transform(data[['Composer']])
    Composer = encodedKeyComposer.toarray()
    Composer_pd = convertEncoderPD(Composer, prefix = 'Composer')   

    encoderArtist = LabelEncoder()
    encodedArtist = encoderArtist.fit_transform(data[['Artist']])
    # encodedArtist = encodedArtist.ravel()
    Artist_pd =  pd.DataFrame(data=encodedArtist, columns= ["Artist"])

    data.drop(['Key','Album_type', 'Composer',  'Artist'], axis=1, inplace=True)

    data = pd.concat([data, key_pd, AlbumType_pd, Composer_pd, Artist_pd], axis=1)


    scaledLoudness = scaleMinMaxLoudness(data[['Loudness']])
    data['Loudness'] = scaledLoudness

    newMinMaxScaler = ['Tempo', 'Duration_ms', 'Views', 'Likes', 'Stream', 'Comments', "Artist"]

    scaler = MinMaxScaler()
    scaledData = scaler.fit_transform(data[newMinMaxScaler])

    for i in range(scaledData.shape[1]):
        data[newMinMaxScaler[i]] = scaledData[:, i]


    return {"key": encoderKey, 'AlbumType': encoderAlbumType, 'Composer': encoderComposer, "Artist":encoderArtist} , scaler, data
    

In [15]:
def createEncodeDataTesting(encoder, scalerStandard, data):

    encoderKey = encoder['key']
    encodedKey = encoderKey.transform(data[['Key']])
    Key = encodedKey.toarray()
    key_pd = convertEncoderPD(Key, prefix = 'key')

    encoderAlbumType = encoder['AlbumType']
    encodedKeyAlbumType = encoderAlbumType.transform(data[['Album_type']])
    AlbumType = encodedKeyAlbumType.toarray()
    AlbumType_pd = convertEncoderPD(AlbumType, prefix = 'AlbumType')

    encoderComposer = encoder['Composer']
    encodedKeyComposer = encoderComposer.transform(data[['Composer']])
    Composer = encodedKeyComposer.toarray()
    Composer_pd = convertEncoderPD(Composer, prefix = 'Composer')   

    encoderArtist = encoder['Artist']
    encodedArtist = encoderArtist.transform(data[['Artist']])
    encodedArtist = encodedArtist.ravel()
    Artist_pd =  pd.DataFrame(data=encodedArtist, columns= ["Artist"])

    data.drop(['Key','Album_type', 'Composer',  'Artist'], axis=1, inplace=True)

    data = pd.concat([data, key_pd, AlbumType_pd, Composer_pd, Artist_pd], axis=1)

    scaledLoudness = scaleMinMaxLoudness(data[['Loudness']])
    data['Loudness'] = scaledLoudness

    newMinMaxScaler = ['Tempo', 'Duration_ms', 'Views', 'Likes', 'Stream', 'Comments', "Artist"]

    scaler = scalerStandard
    scaledData = scaler.transform(data[newMinMaxScaler])

    for i in range(scaledData.shape[1]):
        data[newMinMaxScaler[i]] = scaledData[:, i]

    return data

#### Data Preparation

In [16]:
data = dataPreprocessing(original)

In [17]:
encoders, scalerStandard, scaledData_pd  = createEncodeDataTraining(data)

c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
train_X, test_X, train_Y, test_Y = train_test_split(scaledData_pd, trainingLabels, test_size = 0.20, random_state = 123)

### SVM grid search

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Create the SVM classifier
clf = SVC()

# Create the grid of hyperparameters to search
param_grid = {
    "C": [1, 10, 100],
    "kernel": ['linear', 'polynomial',"rbf"],
    "gamma": [0.001, 0.01, 0.1, 1, 10],
}

# Create the grid search object
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Fit the grid search object to the data
grid_search.fit(train_X, train_Y)

# Print the best parameters
print(grid_search.best_params_)

# Print the best score
print(grid_search.best_score_)

c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
75 fits failed out of a total of 225.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.

{'C': 100, 'gamma': 0.001, 'kernel': 'linear'}
0.19437957724425056


In [31]:
clf1 = SVC(kernel='linear',  gamma=0.001, C=100)
clf1.fit(train_X, train_Y)

filename = "best_SVM.model"
pickle.dump(clf1, open(filename, "wb"))

In [32]:
# load model
loaded_model = pickle.load(open(filename, "rb"))
# you can use loaded model to compute predictions
predicted = loaded_model.predict(test_X)
# predicted = np.rint(predicted)

errors = abs(predicted - test_Y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 2.33


#### Load Testing Dataset

In [33]:
test = pd.read_csv('../html2023-spring-final-project/test.csv')
dataTest = dataPreprocessing(test)
testScaled = createEncodeDataTesting(encoders, scalerStandard, dataTest)

c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#### Testing Danceability

In [34]:
# load model
loaded_model_realTest= pickle.load(open(filename, "rb"))
# you can use loaded model to compute predictions
predictedTest = loaded_model_realTest.predict(testScaled)
predictedTest = np.rint(predictedTest)

In [35]:
label = 17170
idx = []
for i in range(predictedTest.shape[0]):
    idx.append(label + i)
idx = np.array(idx)

In [36]:
testR = np.vstack((idx,predictedTest))
testR = testR.T

In [37]:
predictionTest_pd = pd.DataFrame(data = testR, columns= ['id', 'Danceability'])
predictionTest_pd.to_csv('SVM2.csv')